In [1]:
%%datalabframework getfilename

In [2]:
import datalabframework as dlf
logger = dlf.log.initLogger(__name__, kafka_topic="datalab", kafka_servers="kafka:9092")

2017-06-13 19:26:39,729 - jovyan - __main__ - INFO - init - {'project': {'rootpath': '/home/jovyan/work/notebooks', 'main': 'main.ipynb'}, 'notebook': {'filepath': '/home/jovyan/work/notebooks/flows', 'filename': 'regression.ipynb'}, 'datalab': {'framework': '0.1'}}


In [3]:
DATA_ROOT = '/home/jovyan/work/data'
d = {   
    'input_sample' : 1.0,
    'input_source' : DATA_ROOT + '/titanic/data/raw/train.csv'
}
p = dlf.params.config_fromdict(d)

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(dlf.project.filename()).getOrCreate()
spark.version

'2.1.0'

In [5]:
from extract import extract
from features import features
from models import lr
from validate import validate

importing Jupyter notebook from /home/jovyan/work/notebooks/extract/extract.ipynb
importing Jupyter notebook from /home/jovyan/work/notebooks/features/features.ipynb
importing Jupyter notebook from /home/jovyan/work/notebooks/models/lr.ipynb


In [6]:
df = extract.load_csv(spark, p.input_source)
df = extract.clean_dataset(df)
df = features.inpute_simple(df, 'Embarked')
df = features.impute_lr(df, 'PassengerId', 'Age', ['Sex', 'Pclass', 'SibSp', 'Parch', 'Embarked', 'Fare']) 
df = features.feature_title(df)
df = features.feature_vector(df, 'PassengerId', 'Survived', 
                  ['Pclass','Sex','Age', 'SibSp', 'Parch', 'Fare','Embarked','Title'])

Coefficients: [2.70663137677,-6.46212574775,-3.71767848327,-0.766426906874,0.0,-1.72652572773,-0.00614706540702]
Intercept: 45.19962653687725
numIterations: 11
objectiveHistory: [0.5, 0.4779412071900663, 0.39881579618513935, 0.39619410962365875, 0.39509291906430777, 0.39478668630873737, 0.39469641431808233, 0.3946801657437991, 0.3946687955886411, 0.39466614425208824, 0.3946652407178087]


+-------------------+
|          residuals|
+-------------------+
|-2.7576359629364156|
|  5.144886529400388|
|0.23546619971897087|
| 0.3065868672528964|
|  6.529603206120218|
| 12.874670013768103|
|-14.470868910272241|
| 2.7880416434127078|
|-12.646323655937788|
|-17.226487911191267|
|   19.4257037974276|
| -8.470396793879782|
| 18.222181817838184|
|-11.764969012511845|
|   22.8229780051321|
|-10.703706550476774|
|  9.015076366963651|
|0.17781728242765382|
|-0.9020945678635783|
|-10.763893276065616|
+-------------------+
only showing top 20 rows

RMSE: 12.667974
r2: 0.238445


In [7]:
df.show(5, truncate=False)

+-----------+----------------------------------------------------+-----+
|PassengerId|features                                            |label|
+-----------+----------------------------------------------------+-----+
|1          |(24,[0,1,2,3,5,6,8],[3.0,1.0,22.0,1.0,7.25,1.0,1.0])|0.0  |
|2          |(24,[0,2,3,5,7,10],[1.0,38.0,1.0,71.2833,1.0,1.0])  |1.0  |
|3          |(24,[0,2,5,6,9],[3.0,26.0,7.925,1.0,1.0])           |1.0  |
|4          |(24,[0,2,3,5,6,10],[1.0,35.0,1.0,53.1,1.0,1.0])     |1.0  |
|5          |(24,[0,1,2,5,6,8],[3.0,1.0,35.0,8.05,1.0,1.0])      |0.0  |
+-----------+----------------------------------------------------+-----+
only showing top 5 rows



In [8]:
model = lr.lr_model(df)
metric_name, metric = validate.auc(df,model)

In [9]:
print("Evaluation {} : {}".format(metric_name, metric))

Evaluation areaUnderROC : 0.8742077567933189
